In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch10SQLPyspark").enableHiveSupport().getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("ch08Joins").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_17801


In [5]:
#data_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data"
data_dir = "/user/kranthidr/dataSets/spark-guide"
data_path = data_dir + "/flight-data/json/2015-summary.json"

In [6]:
spark.read.json(data_path).createOrReplaceTempView("some_sql_view") 

In [7]:
spark.sql("SELECT DEST_COUNTRY_NAME, sum(count) FROM some_sql_view GROUP BY DEST_COUNTRY_NAME").show(10)

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|         Anguilla|        41|
|           Russia|       176|
|         Paraguay|        60|
|          Senegal|        40|
|           Sweden|       118|
|         Kiribati|        26|
|           Guyana|        64|
|      Philippines|       134|
|         Djibouti|         1|
|         Malaysia|         2|
+-----------------+----------+
only showing top 10 rows



In [8]:
spark.sql("SELECT DEST_COUNTRY_NAME, sum(count) FROM some_sql_view GROUP BY DEST_COUNTRY_NAME")\
.where("DEST_COUNTRY_NAME like 'S%'")\
.where("`sum(count)` > 10").show(10) 

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|             Senegal|        40|
|              Sweden|       118|
|               Spain|       420|
|    Saint Barthelemy|        39|
|Saint Kitts and N...|       139|
|         South Korea|      1048|
|        Sint Maarten|       325|
|        Saudi Arabia|        83|
|         Switzerland|       294|
|         Saint Lucia|       123|
+--------------------+----------+
only showing top 10 rows



In [9]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS Kranthidr_db
LOCATION '/user/kranthidr/Kranthidr_db'
""").show()

++
||
++
++



In [10]:
spark.sql("""
USE Kranthidr_db
""").show()

++
||
++
++



In [11]:
spark.sql("""
CREATE TABLE IF NOT EXISTS flights (DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path '/user/kranthidr/dataSets/spark-guide/flight-data/json/2015-summary.json')
""")

DataFrame[]

In [12]:
spark.sql("""
select * from flights limit 10
""").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [13]:
spark.sql("""
CREATE TABLE flights_csv (
  DEST_COUNTRY_NAME STRING,
  ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
  count LONG)
USING csv OPTIONS (header true, path '/user/kranthidr/dataSets/spark-guide/flight-data/csv/2015-summary.csv')
""")

DataFrame[]

In [14]:
spark.sql("""
select * from flights_csv limit 10
""").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [15]:
spark.sql("""
show databases
""").show()

+--------------------+
|        databaseName|
+--------------------+
|   000000_abhi_cards|
|            00_cards|
|              00_db1|
|           05121990a|
|        0_ganesh1146|
|          0_practice|
|            0_subrat|
|          10bq1a0410|
|               10feb|
|          1pakr12345|
|                1ssm|
|23singhabhishek_s...|
|             24thmay|
|                2jun|
|               30may|
|               31may|
|              5avril|
|                5jun|
|             5junraj|
|         5junrajnish|
+--------------------+
only showing top 20 rows



In [25]:
spark.sql("""
show tables
""").show()

+------------+-------------------+-----------+
|    database|          tableName|isTemporary|
+------------+-------------------+-----------+
|kranthidr_db|            flights|      false|
|kranthidr_db|        flights_csv|      false|
|kranthidr_db|flights_from_select|      false|
|kranthidr_db|       hive_flights|      false|
|kranthidr_db|partitioned_flights|      false|
|            |      some_sql_view|       true|
+------------+-------------------+-----------+



In [17]:
spark.sql("""
CREATE TABLE IF NOT EXISTS flights_from_select
  AS SELECT * FROM flights
""").show()


++
||
++
++



In [18]:
spark.sql("""
SELECT * FROM flights_from_select
""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [19]:
spark.sql("""
CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5
""").show()

++
||
++
++



In [23]:
data_dir+"/flight-data-hive/"

'/user/kranthidr/dataSets/spark-guide/flight-data-hive/'

In [24]:
spark.sql("""
CREATE EXTERNAL TABLE hive_flights (
  DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/user/kranthidr/dataSets/spark-guide/flight-data-hive/'
""").show()

++
||
++
++



In [27]:
spark.sql("""
CREATE EXTERNAL TABLE hive_flights_2
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION '/user/kranthidr/dataSets/spark-guide/flight-data-hive/' AS SELECT * FROM flights
""").show()


++
||
++
++



In [28]:
spark.sql("""
show tables
""").show()

+------------+-------------------+-----------+
|    database|          tableName|isTemporary|
+------------+-------------------+-----------+
|kranthidr_db|            flights|      false|
|kranthidr_db|        flights_csv|      false|
|kranthidr_db|flights_from_select|      false|
|kranthidr_db|       hive_flights|      false|
|kranthidr_db|     hive_flights_2|      false|
|kranthidr_db|partitioned_flights|      false|
|            |      some_sql_view|       true|
+------------+-------------------+-----------+



In [29]:
spark.sql("""
INSERT INTO flights_from_select
  SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20
""").show()

++
||
++
++



In [30]:
spark.sql("""
INSERT INTO partitioned_flights
  PARTITION (DEST_COUNTRY_NAME="UNITED STATES")
  SELECT count, ORIGIN_COUNTRY_NAME FROM flights
  WHERE DEST_COUNTRY_NAME='UNITED STATES' LIMIT 12
""").show()

++
||
++
++



In [31]:
spark.sql("""
DESCRIBE TABLE flights_csv
""").show()

+-------------------+---------+--------------------+
|           col_name|data_type|             comment|
+-------------------+---------+--------------------+
|  DEST_COUNTRY_NAME|   string|                null|
|ORIGIN_COUNTRY_NAME|   string|remember, the US ...|
|              count|   bigint|                null|
+-------------------+---------+--------------------+



In [34]:
spark.sql("""
SHOW PARTITIONS partitioned_flights
""").show(20, False)

+-------------------------------+
|partition                      |
+-------------------------------+
|DEST_COUNTRY_NAME=Egypt        |
|DEST_COUNTRY_NAME=United States|
+-------------------------------+



In [35]:
spark.sql("""
REFRESH table partitioned_flights
""").show(20, False)

++
||
++
++



In [36]:
spark.sql("""
MSCK REPAIR TABLE partitioned_flights
""").show(20, False)

++
||
++
++



In [39]:
spark.sql("""
DROP TABLE flights_csv
""").show()

++
||
++
++



In [41]:
spark.sql("""
DROP TABLE IF EXISTS flights_csv
""").show()

++
||
++
++



In [42]:
spark.sql("""
CACHE TABLE flights
""").show()

++
||
++
++



In [43]:
spark.sql("""
UNCACHE TABLE FLIGHTS
""").show()

++
||
++
++



In [47]:
spark.sql("""
CREATE VIEW IF NOT EXISTS just_usa_view AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

++
||
++
++



In [48]:
spark.sql("""
CREATE TEMP VIEW just_usa_view_temp AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

++
||
++
++



In [49]:
spark.sql("""
CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

++
||
++
++



In [50]:
spark.sql("""
SHOW TABLES
""").show()

+------------+-------------------+-----------+
|    database|          tableName|isTemporary|
+------------+-------------------+-----------+
|kranthidr_db|            flights|      false|
|kranthidr_db|flights_from_select|      false|
|kranthidr_db|       hive_flights|      false|
|kranthidr_db|     hive_flights_2|      false|
|kranthidr_db|      just_usa_view|      false|
|kranthidr_db|partitioned_flights|      false|
|            | just_usa_view_temp|       true|
|            |      some_sql_view|       true|
+------------+-------------------+-----------+



In [51]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'
""").show()

++
||
++
++



In [52]:
spark.sql("""
SELECT * FROM just_usa_view_temp
""").show()

+-----------------+--------------------+-----+
|DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+-----------------+--------------------+-----+
|    United States|             Romania|   15|
|    United States|             Croatia|    1|
|    United States|             Ireland|  344|
|    United States|               India|   62|
|    United States|           Singapore|    1|
|    United States|             Grenada|   62|
|    United States|        Sint Maarten|  325|
|    United States|    Marshall Islands|   39|
|    United States|            Paraguay|    6|
|    United States|           Gibraltar|    1|
|    United States|Federated States ...|   69|
|    United States|              Russia|  161|
|    United States|         Netherlands|  660|
|    United States|             Senegal|   42|
|    United States|              Angola|   13|
|    United States|            Anguilla|   38|
|    United States|             Ecuador|  300|
|    United States|              Cyprus|    1|
|    United S

In [55]:
spark.sql("""
EXPLAIN SELECT * FROM just_usa_view
""").take(2)

[Row(plan=u'== Physical Plan ==\n*(1) Project [DEST_COUNTRY_NAME#52, ORIGIN_COUNTRY_NAME#53, count#54L]\n+- *(1) Filter (isnotnull(dest_country_name#52) && (dest_country_name#52 = United States))\n   +- *(1) FileScan json kranthidr_db.flights[DEST_COUNTRY_NAME#52,ORIGIN_COUNTRY_NAME#53,count#54L] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/flight-data/..., PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>')]

In [57]:
spark.sql("""
EXPLAIN SELECT * FROM flights WHERE dest_country_name = 'United States'
""").head()

Row(plan=u'== Physical Plan ==\n*(1) Project [DEST_COUNTRY_NAME#52, ORIGIN_COUNTRY_NAME#53, count#54L]\n+- *(1) Filter (isnotnull(dest_country_name#52) && (dest_country_name#52 = United States))\n   +- *(1) FileScan json kranthidr_db.flights[DEST_COUNTRY_NAME#52,ORIGIN_COUNTRY_NAME#53,count#54L] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/flight-data/..., PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>')

In [58]:
spark.sql("""
DROP VIEW IF EXISTS just_usa_view
""").show()

++
||
++
++



In [61]:
spark.sql("""
SHOW DATABASES 'kr*'
""").show()

+--------------------+
|        databaseName|
+--------------------+
|    kr_arun_problem6|
|     kr_sqoop_import|
|        kranthidr_db|
|       kranthikarnam|
|              kranti|
|               krish|
|            krish_db|
| krish_retail_db_txt|
|              krishm|
|           krishmani|
|             krishna|
|          krishna324|
|krishna324_daily_...|
|      krishna324_orc|
|krishna324_retail...|
|          krishna533|
|krishna533_sqoop_...|
| krishna_201711_demo|
| krishna_arun_201801|
| krishna_arun_201802|
+--------------------+
only showing top 20 rows



In [62]:
spark.sql("""
SHOW DATABASES
""").count()

7010

In [63]:
spark.sql("""
SELECT * FROM default.flights
""").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
+-----------------+-------------------+-----+



In [64]:
spark.sql("""
SELECT current_database()
""").show()

+------------------+
|current_database()|
+------------------+
|      kranthidr_db|
+------------------+



In [66]:
# spark.sql("""
# DROP DATABASE IF EXISTS kranthidr_db
# """).show()

# AnalysisException: u'org.apache.hadoop.hive.ql.metadata.HiveException: 
#     InvalidOperationException(message:Database kranthidr_db is not empty. One or more tables exist.);'


In [ ]:
# SELECT [ALL|DISTINCT] named_expression[, named_expression, ...]
#     FROM relation[, relation, ...]
#     [lateral_view[, lateral_view, ...]]
#     [WHERE boolean_expression]
#     [aggregation [HAVING boolean_expression]]
#     [ORDER BY sort_expressions]
#     [CLUSTER BY expressions]
#     [DISTRIBUTE BY expressions]
#     [SORT BY sort_expressions]
#     [WINDOW named_window[, WINDOW named_window, ...]]
#     [LIMIT num_rows]

# named_expression:
#     : expression [AS alias]

# relation:
#     | join_relation
#     | (table_name|query|relation) [sample] [AS alias]
#     : VALUES (expressions)[, (expressions), ...]
#           [AS (column_name[, column_name, ...])]

# expressions:
#     : expression[, expression, ...]

# sort_expressions:
#     : expression [ASC|DESC][, expression [ASC|DESC], ...]

In [67]:
spark.sql("""
SELECT
  CASE WHEN DEST_COUNTRY_NAME = 'UNITED STATES' THEN 1
       WHEN DEST_COUNTRY_NAME = 'Egypt' THEN 0
       ELSE -1 END
FROM partitioned_flights
""").show()

+--------------------------------------------------------------------------------------------------------+
|CASE WHEN (DEST_COUNTRY_NAME = UNITED STATES) THEN 1 WHEN (DEST_COUNTRY_NAME = Egypt) THEN 0 ELSE -1 END|
+--------------------------------------------------------------------------------------------------------+
|                                                                                                      -1|
|                                                                                                      -1|
|                                                                                                      -1|
|                                                                                                      -1|
|                                                                                                       0|
+--------------------------------------------------------------------------------------------------------+



In [68]:
spark.sql("""
CREATE VIEW IF NOT EXISTS nested_data AS
  SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights
""").show()

++
||
++
++



In [70]:
spark.sql("""
SELECT * FROM nested_data
""").show(3, False)

+------------------------+-----+
|country                 |count|
+------------------------+-----+
|[United States, Romania]|15   |
|[United States, Croatia]|1    |
|[United States, Ireland]|344  |
+------------------------+-----+
only showing top 3 rows



In [72]:
spark.sql("""
SELECT country.DEST_COUNTRY_NAME, count FROM nested_data
""").show(5, False)

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|United States    |15   |
|United States    |1    |
|United States    |344  |
|Egypt            |15   |
|United States    |62   |
+-----------------+-----+
only showing top 5 rows



In [73]:
spark.sql("""
SELECT country.*, count FROM nested_data
""").show(5, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [75]:
spark.sql("""
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_counts,
  collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM flights GROUP BY DEST_COUNTRY_NAME
""").show(5, False)

+--------+-------------+---------------+
|new_name|flight_counts|origin_set     |
+--------+-------------+---------------+
|Anguilla|[41]         |[United States]|
|Paraguay|[60]         |[United States]|
|Russia  |[176]        |[United States]|
|Senegal |[40]         |[United States]|
|Sweden  |[118]        |[United States]|
+--------+-------------+---------------+
only showing top 5 rows



In [78]:
spark.sql("""
SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights
""").show(5, False)

+-----------------+--------------+
|DEST_COUNTRY_NAME|array(1, 2, 3)|
+-----------------+--------------+
|United States    |[1, 2, 3]     |
|United States    |[1, 2, 3]     |
|United States    |[1, 2, 3]     |
|Egypt            |[1, 2, 3]     |
|United States    |[1, 2, 3]     |
+-----------------+--------------+
only showing top 5 rows



In [79]:
spark.sql("""
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count)[0]
FROM flights GROUP BY DEST_COUNTRY_NAME
""").show(5, False)

+--------+----------------------+
|new_name|collect_list(count)[0]|
+--------+----------------------+
|Anguilla|41                    |
|Paraguay|60                    |
|Russia  |176                   |
|Senegal |40                    |
|Sweden  |118                   |
+--------+----------------------+
only showing top 5 rows



In [80]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW flights_agg AS
  SELECT DEST_COUNTRY_NAME, collect_list(count) as collected_counts
  FROM flights GROUP BY DEST_COUNTRY_NAME
""").show()

++
||
++
++



In [82]:
spark.sql("""
SELECT explode(collected_counts), DEST_COUNTRY_NAME FROM flights_agg
""").show(5, False)

+---+-----------------+
|col|DEST_COUNTRY_NAME|
+---+-----------------+
|41 |Anguilla         |
|60 |Paraguay         |
|176|Russia           |
|40 |Senegal          |
|118|Sweden           |
+---+-----------------+
only showing top 5 rows



In [84]:
spark.sql("""
SHOW FUNCTIONS
""").show()

+--------------------+
|            function|
+--------------------+
|                   !|
|                   %|
|                   &|
|                   *|
|                   +|
|                   -|
|                   /|
|                   <|
|                  <=|
|                 <=>|
|                   =|
|                  ==|
|                   >|
|                  >=|
|                   ^|
|                 abs|
|                acos|
|          add_months|
|                 and|
|approx_count_dist...|
+--------------------+
only showing top 20 rows



In [86]:
spark.sql("""
SHOW SYSTEM FUNCTIONS
""").show()

+--------------------+
|            function|
+--------------------+
|                   !|
|                   %|
|                   &|
|                   *|
|                   +|
|                   -|
|                   /|
|                   <|
|                  <=|
|                 <=>|
|                   =|
|                  ==|
|                   >|
|                  >=|
|                   ^|
|                 abs|
|                acos|
|          add_months|
|                 and|
|approx_count_dist...|
+--------------------+
only showing top 20 rows



In [89]:
spark.sql("""
SHOW USER FUNCTIONS
""").show()

+--------+
|function|
+--------+
+--------+



In [91]:
spark.sql("""
SHOW FUNCTIONS "s*"
""").show()

+------------------+
|          function|
+------------------+
|            second|
|         sentences|
|               sha|
|              sha1|
|              sha2|
|         shiftleft|
|        shiftright|
|shiftrightunsigned|
|              sign|
|            signum|
|               sin|
|              sinh|
|              size|
|          skewness|
|          smallint|
|        sort_array|
|           soundex|
|             space|
|spark_partition_id|
|             split|
+------------------+
only showing top 20 rows



In [92]:
spark.sql("""
SHOW FUNCTIONS LIKE "collect*"
""").show()



+------------+
|    function|
+------------+
|collect_list|
| collect_set|
+------------+



In [96]:
# spark.sql("""
# SELECT count, power3(count) FROM flights
# """).show()
# AnalysisException: u"Undefined function: 'power3'. 
#     This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; 
#     line 2 pos 14"

In [97]:
spark.sql("""
select current_database()
""").show()

+------------------+
|current_database()|
+------------------+
|      kranthidr_db|
+------------------+



In [94]:
spark.sql("""
SELECT dest_country_name FROM flights
GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5
""").show()

+-----------------+
|dest_country_name|
+-----------------+
|    United States|
|           Canada|
|           Mexico|
|   United Kingdom|
|            Japan|
+-----------------+



In [95]:
spark.sql("""
SELECT * FROM flights
WHERE origin_country_name IN (SELECT dest_country_name FROM flights
      GROUP BY dest_country_name ORDER BY sum(count) DESC LIMIT 5)
""").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Egypt|      United States|   15|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|Saint Vincent and...|      United States|    1|
|               Italy|      United States|  382|
|            Pakistan|      United States|   12|
|             Iceland|      United States|  181|
|    Marshall Islands|      United States|   42|
|          Luxembourg|      United States|  155|
|            Honduras|      United States|  362|
|         The Bahama

In [98]:
spark.sql("""
SELECT * FROM flights f1
WHERE EXISTS (SELECT 1 FROM flights f2
            WHERE f1.dest_country_name = f2.origin_country_name)
AND EXISTS (SELECT 1 FROM flights f2
            WHERE f2.dest_country_name = f1.origin_country_name)
""").show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|               Malta|       United States|    1|
|            Anguilla|       United States|   41|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|Turks and Caicos ...|       United States|  230|


In [99]:
spark.sql("""
SELECT *, (SELECT max(count) FROM flights) AS maximum FROM flights
""").show()

+--------------------+-------------------+-----+-------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|maximum|
+--------------------+-------------------+-----+-------+
|       United States|            Romania|   15| 370002|
|       United States|            Croatia|    1| 370002|
|       United States|            Ireland|  344| 370002|
|               Egypt|      United States|   15| 370002|
|       United States|              India|   62| 370002|
|       United States|          Singapore|    1| 370002|
|       United States|            Grenada|   62| 370002|
|          Costa Rica|      United States|  588| 370002|
|             Senegal|      United States|   40| 370002|
|             Moldova|      United States|    1| 370002|
|       United States|       Sint Maarten|  325| 370002|
|       United States|   Marshall Islands|   39| 370002|
|              Guyana|      United States|   64| 370002|
|               Malta|      United States|    1| 370002|
|            Anguilla|      Uni

In [100]:
spark.sql("""
SET spark.sql.shuffle.partitions=20
""").show()

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.shuffle...|   20|
+--------------------+-----+

